# Reference materials
0. [Vector Representations of Words](https://www.tensorflow.org/tutorials/word2vec)
1. [Word2Vec Tutorial Part 1 - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
2. [Word2Vec Tutorial Part 2 - Negative Sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)
3. [From word2vec to doc2vec: an approach driven by Chinese restaurant process](https://medium.com/kifi-engineering/from-word2vec-to-doc2vec-an-approach-driven-by-chinese-restaurant-process-93d3602eaa31)
4. [Doc2Vec tutorial using Gensim](https://medium.com/@klintcho/doc2vec-tutorial-using-gensim-ab3ac03d3a1)
5. [Sentiment Analysis Using Doc2Vec](http://linanqiu.github.io/2015/10/07/word2vec-sentiment/)
6. [Understanding Convolutional Neural Networks for NLP](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)
7. [Implementing a CNN for Text Classification in TensorFlow](http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/)

# How to do document clustering with word2vec and doc2vec?

**Steps:**
1. Train a word2vec model to get word vectors
    - Made word2vec_starter.py to work
    - Question: How to feed the embeddings to the doc2vec model?
2. Get doc2vec from word2vec, but HOW(in Tensorflow)???
    - Chinese Restaurant Process method
    - Gensim doc2vec
    - CNN with multi-genre classification(See Viola's work)
3. Clustering
    - Feed document vectors as features to a classification model
    - Calculate cosine similarity between document vectors, maybe cluster documents using the DBSCAN method?